In [3]:
# ! pip install PyPDF2
# ! pip install docx
# ! pip install python-docx
import os
import sqlite3
import hashlib
import datetime
import shutil
import difflib
from pathlib import Path
import PyPDF2
import docx
import pandas as pd
from typing import Dict, List, Tuple, Optional, Union


class DocumentManager:
    """
    Gestor de documentos normativos para la creación y actualización de una base de datos
    que servirá como fuente para un sistema RAG (Retrieval-Augmented Generation).
    """
    
    def __init__(self, db_path: str = "normativa_upv.db", docs_dir: str = "documentos"):
        """
        Inicializa el gestor de documentos.
        
        Args:
            db_path: Ruta al archivo de la base de datos SQLite
            docs_dir: Directorio donde se almacenan los documentos
        """
        self.db_path = db_path
        self.docs_dir = docs_dir
        self.conn = None
        self.cursor = None
        
        # Asegurar que el directorio de documentos existe
        os.makedirs(self.docs_dir, exist_ok=True)
        
        # Conexión a la base de datos
        self._connect_db()
        
        # Crear tablas si no existen
        self._create_tables()
    
    def _connect_db(self):
        """Establece conexión con la base de datos SQLite."""
        self.conn = sqlite3.connect(self.db_path)
        self.cursor = self.conn.cursor()
    
    def _create_tables(self):
        """Crea las tablas necesarias en la base de datos si no existen."""
        # Tabla de documentos
        self.cursor.execute('''
        CREATE TABLE IF NOT EXISTS documentos (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nombre TEXT NOT NULL,
            ruta TEXT NOT NULL,
            categoria TEXT NOT NULL,
            hash TEXT NOT NULL,
            fecha_modificacion TEXT NOT NULL,
            fecha_indexacion TEXT NOT NULL,
            tamanio INTEGER NOT NULL,
            version INTEGER DEFAULT 1,
            UNIQUE(ruta)
        )
        ''')
        
        # Tabla de versiones de documentos
        self.cursor.execute('''
        CREATE TABLE IF NOT EXISTS versiones_documentos (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            doc_id INTEGER NOT NULL,
            hash TEXT NOT NULL,
            fecha_modificacion TEXT NOT NULL,
            fecha_indexacion TEXT NOT NULL,
            version INTEGER NOT NULL,
            cambios TEXT,
            FOREIGN KEY (doc_id) REFERENCES documentos(id)
        )
        ''')
        
        # Tabla para almacenar el contenido procesado de los documentos
        self.cursor.execute('''
        CREATE TABLE IF NOT EXISTS contenido_documentos (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            doc_id INTEGER NOT NULL,
            seccion TEXT,
            contenido TEXT NOT NULL,
            embedding_id TEXT,
            FOREIGN KEY (doc_id) REFERENCES documentos(id)
        )
        ''')
        
        self.conn.commit()
    
    def calcular_hash_archivo(self, ruta_archivo: str) -> str:
        """
        Calcula el hash SHA-256 de un archivo para identificar cambios.
        
        Args:
            ruta_archivo: Ruta al archivo
            
        Returns:
            Hash SHA-256 del archivo
        """
        hash_sha256 = hashlib.sha256()
        
        with open(ruta_archivo, "rb") as f:
            # Leer el archivo en bloques para manejar archivos grandes
            for bloque in iter(lambda: f.read(4096), b""):
                hash_sha256.update(bloque)
                
        return hash_sha256.hexdigest()
    
    def obtener_info_documento(self, ruta_archivo: str) -> Dict:
        """
        Obtiene información básica de un documento.
        
        Args:
            ruta_archivo: Ruta al archivo
            
        Returns:
            Diccionario con información del documento
        """
        archivo = Path(ruta_archivo)
        stats = archivo.stat()
        
        return {
            "nombre": archivo.name,
            "ruta": str(archivo.absolute()),
            "categoria": archivo.parent.name,
            "hash": self.calcular_hash_archivo(ruta_archivo),
            "fecha_modificacion": datetime.datetime.fromtimestamp(stats.st_mtime).isoformat(),
            "fecha_indexacion": datetime.datetime.now().isoformat(),
            "tamanio": stats.st_size
        }
    
    def extraer_texto(self, ruta_archivo: str) -> str:
        """
        Extrae el texto de un documento según su tipo.
        
        Args:
            ruta_archivo: Ruta al archivo
            
        Returns:
            Texto extraído del documento
        """
        extension = Path(ruta_archivo).suffix.lower()
        
        if extension == ".pdf":
            return self._extraer_texto_pdf(ruta_archivo)
        elif extension in [".docx", ".doc"]:
            return self._extraer_texto_docx(ruta_archivo)
        elif extension == ".txt":
            return self._extraer_texto_txt(ruta_archivo)
        else:
            return f"Formato no soportado: {extension}"
    
    def _extraer_texto_pdf(self, ruta_archivo: str) -> str:
        """Extrae texto de un archivo PDF."""
        texto = ""
        try:
            with open(ruta_archivo, "rb") as archivo:
                lector_pdf = PyPDF2.PdfReader(archivo)
                for pagina in lector_pdf.pages:
                    texto += pagina.extract_text() + "\n"
        except Exception as e:
            texto = f"Error al procesar PDF: {str(e)}"
        return texto
    
    def _extraer_texto_docx(self, ruta_archivo: str) -> str:
        """Extrae texto de un archivo DOCX."""
        texto = ""
        try:
            doc = docx.Document(ruta_archivo)
            for parrafo in doc.paragraphs:
                texto += parrafo.text + "\n"
        except Exception as e:
            texto = f"Error al procesar DOCX: {str(e)}"
        return texto
    
    def _extraer_texto_txt(self, ruta_archivo: str) -> str:
        """Extrae texto de un archivo TXT."""
        try:
            with open(ruta_archivo, "r", encoding="utf-8") as archivo:
                return archivo.read()
        except UnicodeDecodeError:
            # Intentar con otra codificación si utf-8 falla
            try:
                with open(ruta_archivo, "r", encoding="latin-1") as archivo:
                    return archivo.read()
            except Exception as e:
                return f"Error al procesar TXT: {str(e)}"
    
    def indexar_documento(self, ruta_archivo: str) -> int:
        """
        Indexa un nuevo documento en la base de datos.
        
        Args:
            ruta_archivo: Ruta al archivo
            
        Returns:
            ID del documento indexado
        """
        info_doc = self.obtener_info_documento(ruta_archivo)
        
        # Insertar información básica del documento
        self.cursor.execute('''
        INSERT INTO documentos 
        (nombre, ruta, categoria, hash, fecha_modificacion, fecha_indexacion, tamanio, version)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            info_doc["nombre"],
            info_doc["ruta"],
            info_doc["categoria"],
            info_doc["hash"],
            info_doc["fecha_modificacion"],
            info_doc["fecha_indexacion"],
            info_doc["tamanio"],
            1
        ))
        
        doc_id = self.cursor.lastrowid
        
        # Extraer y almacenar el contenido del documento
        texto = self.extraer_texto(ruta_archivo)
        
        # Aquí se podría implementar una lógica más avanzada para dividir el texto en secciones
        # Por ahora, almacenamos todo el contenido como una sola sección
        self.cursor.execute('''
        INSERT INTO contenido_documentos (doc_id, seccion, contenido)
        VALUES (?, ?, ?)
        ''', (doc_id, "completo", texto))
        
        self.conn.commit()
        return doc_id
    
    def actualizar_documento(self, doc_id: int, ruta_archivo: str) -> bool:
        """
        Actualiza un documento existente en la base de datos.
        
        Args:
            doc_id: ID del documento en la base de datos
            ruta_archivo: Ruta al archivo actualizado
            
        Returns:
            True si se actualizó, False en caso contrario
        """
        # Obtener información del documento actual en la base de datos
        self.cursor.execute("SELECT hash, version FROM documentos WHERE id = ?", (doc_id,))
        resultado = self.cursor.fetchone()
        
        if not resultado:
            return False
        
        hash_antiguo, version_actual = resultado
        
        # Calcular el hash del archivo actualizado
        info_doc = self.obtener_info_documento(ruta_archivo)
        hash_nuevo = info_doc["hash"]
        
        # Si el hash no ha cambiado, no es necesario actualizar
        if hash_antiguo == hash_nuevo:
            return False
        
        # Guardar la versión anterior
        self.cursor.execute('''
        INSERT INTO versiones_documentos 
        (doc_id, hash, fecha_modificacion, fecha_indexacion, version, cambios)
        SELECT id, hash, fecha_modificacion, fecha_indexacion, version, NULL
        FROM documentos WHERE id = ?
        ''', (doc_id,))
        
        # Extraer contenido del documento actualizado
        texto_nuevo = self.extraer_texto(ruta_archivo)
        
        # Obtener contenido anterior para comparación
        self.cursor.execute("SELECT contenido FROM contenido_documentos WHERE doc_id = ? AND seccion = ?", 
                           (doc_id, "completo"))
        resultado = self.cursor.fetchone()
        texto_antiguo = resultado[0] if resultado else ""
        
        # Calcular diferencias
        diff = difflib.unified_diff(
            texto_antiguo.splitlines(),
            texto_nuevo.splitlines(),
            fromfile='version anterior',
            tofile='version nueva',
            lineterm=''
        )
        cambios = '\n'.join(list(diff))
        
        # Actualizar la información del documento
        self.cursor.execute('''
        UPDATE documentos 
        SET hash = ?, fecha_modificacion = ?, fecha_indexacion = ?, version = version + 1
        WHERE id = ?
        ''', (
            hash_nuevo,
            info_doc["fecha_modificacion"],
            info_doc["fecha_indexacion"],
            doc_id
        ))
        
        # Actualizar la entrada de versiones con los cambios detectados
        self.cursor.execute('''
        UPDATE versiones_documentos 
        SET cambios = ?
        WHERE doc_id = ? AND version = ?
        ''', (cambios, doc_id, version_actual))
        
        # Actualizar el contenido
        self.cursor.execute('''
        UPDATE contenido_documentos 
        SET contenido = ?
        WHERE doc_id = ? AND seccion = ?
        ''', (texto_nuevo, doc_id, "completo"))
        
        self.conn.commit()
        return True
    
    def obtener_documento_por_ruta(self, ruta_archivo: str) -> Optional[Dict]:
        """
        Busca un documento en la base de datos por su ruta.
        
        Args:
            ruta_archivo: Ruta al archivo
            
        Returns:
            Información del documento o None si no existe
        """
        ruta_absoluta = str(Path(ruta_archivo).absolute())
        
        self.cursor.execute('''
        SELECT id, nombre, categoria, hash, fecha_modificacion, fecha_indexacion, tamanio, version
        FROM documentos WHERE ruta = ?
        ''', (ruta_absoluta,))
        
        resultado = self.cursor.fetchone()
        
        if not resultado:
            return None
        
        return {
            "id": resultado[0],
            "nombre": resultado[1],
            "categoria": resultado[2],
            "hash": resultado[3],
            "fecha_modificacion": resultado[4],
            "fecha_indexacion": resultado[5],
            "tamanio": resultado[6],
            "version": resultado[7]
        }
    
    def procesar_directorio(self, directorio: str, actualizar: bool = False) -> Dict:
        """
        Procesa todos los documentos en un directorio y sus subdirectorios.
        
        Args:
            directorio: Ruta al directorio
            actualizar: Si es True, actualiza documentos existentes
            
        Returns:
            Estadísticas del proceso
        """
        if not os.path.exists(directorio):
            return {"error": f"El directorio {directorio} no existe"}
        
        stats = {
            "nuevos": 0,
            "actualizados": 0,
            "sin_cambios": 0,
            "errores": 0
        }
        
        # Extensiones de archivo soportadas
        extensiones = ['.pdf', '.docx', '.doc', '.txt']
        
        # Recorrer todos los archivos del directorio y subdirectorios
        for ruta_actual, _, archivos in os.walk(directorio):
            for archivo in archivos:
                extension = os.path.splitext(archivo)[1].lower()
                
                if extension not in extensiones:
                    continue
                
                ruta_completa = os.path.join(ruta_actual, archivo)
                
                try:
                    # Verificar si el documento ya existe en la base de datos
                    documento_existente = self.obtener_documento_por_ruta(ruta_completa)
                    
                    if documento_existente:
                        if actualizar:
                            # Intentar actualizar el documento
                            actualizado = self.actualizar_documento(documento_existente["id"], ruta_completa)
                            if actualizado:
                                stats["actualizados"] += 1
                            else:
                                stats["sin_cambios"] += 1
                        else:
                            stats["sin_cambios"] += 1
                    else:
                        # Documento nuevo, indexarlo
                        self.indexar_documento(ruta_completa)
                        stats["nuevos"] += 1
                except Exception as e:
                    print(f"Error al procesar {ruta_completa}: {str(e)}")
                    stats["errores"] += 1
        
        return stats
    
    def buscar_documentos(self, termino: str = None, categoria: str = None) -> List[Dict]:
        """
        Busca documentos en la base de datos.
        
        Args:
            termino: Término a buscar en el nombre
            categoria: Categoría para filtrar
            
        Returns:
            Lista de documentos que coinciden con los criterios
        """
        consulta = "SELECT id, nombre, categoria, fecha_modificacion, version FROM documentos WHERE 1=1"
        parametros = []
        
        if termino:
            consulta += " AND nombre LIKE ?"
            parametros.append(f"%{termino}%")
        
        if categoria:
            consulta += " AND categoria = ?"
            parametros.append(categoria)
        
        self.cursor.execute(consulta, parametros)
        resultados = self.cursor.fetchall()
        
        documentos = []
        for resultado in resultados:
            documentos.append({
                "id": resultado[0],
                "nombre": resultado[1],
                "categoria": resultado[2],
                "fecha_modificacion": resultado[3],
                "version": resultado[4]
            })
        
        return documentos
    
    def obtener_categorias(self) -> List[str]:
        """
        Obtiene todas las categorías de documentos existentes.
        
        Returns:
            Lista de categorías
        """
        self.cursor.execute("SELECT DISTINCT categoria FROM documentos")
        return [row[0] for row in self.cursor.fetchall()]
    
    def generar_informe(self) -> pd.DataFrame:
        """
        Genera un informe detallado de todos los documentos.
        
        Returns:
            DataFrame con información de los documentos
        """
        consulta = """
        SELECT d.id, d.nombre, d.categoria, d.fecha_modificacion, 
               d.fecha_indexacion, d.tamanio, d.version,
               COUNT(v.id) as versiones_anteriores
        FROM documentos d
        LEFT JOIN versiones_documentos v ON d.id = v.doc_id
        GROUP BY d.id
        ORDER BY d.categoria, d.nombre
        """
        
        self.cursor.execute(consulta)
        resultados = self.cursor.fetchall()
        
        columnas = [
            "ID", "Nombre", "Categoría", "Fecha Modificación", 
            "Fecha Indexación", "Tamaño (bytes)", "Versión Actual", 
            "Versiones Anteriores"
        ]
        
        df = pd.DataFrame(resultados, columns=columnas)
        return df
    
    def cerrar(self):
        """Cierra la conexión a la base de datos."""
        if self.conn:
            self.conn.close()


# Función para crear la estructura de directorios y archivos de prueba
def crear_estructura_dummy():
    """
    Crea una estructura de directorios y archivos de ejemplo para probar el sistema.
    """
    # Crear directorios
    directorios = [
        "documentos/normativa_masters",
        "documentos/grados_normativa",
        "documentos/erasmus_normativa"
    ]
    
    for directorio in directorios:
        os.makedirs(directorio, exist_ok=True)
    
    # Crear archivos de texto con contenido de ejemplo
    archivos = {
        "documentos/normativa_masters/master_informatica.txt": 
            "Normativa del Máster en Ingeniería Informática\n\n"
            "1. Requisitos de admisión\n"
            "Para acceder al máster es necesario tener un título de grado en informática.\n\n"
            "2. Plan de estudios\n"
            "El plan de estudios consta de 90 ECTS distribuidos en 3 semestres.",
            
        "documentos/normativa_masters/master_teleco.txt":
            "Normativa del Máster en Ingeniería de Telecomunicación\n\n"
            "1. Requisitos de admisión\n"
            "Se requiere grado en telecomunicaciones o similar.\n\n"
            "2. Asignaturas obligatorias\n"
            "Redes avanzadas, Sistemas de comunicación, Tratamiento de señal.",
            
        "documentos/grados_normativa/grado_ade.txt":
            "Normativa del Grado en Administración y Dirección de Empresas\n\n"
            "1. Plan de estudios\n"
            "El grado consta de 240 ECTS distribuidos en 4 años.\n\n"
            "2. Prácticas en empresa\n"
            "Las prácticas son obligatorias y tienen una duración de 300 horas.",
            
        "documentos/erasmus_normativa/requisitos_erasmus.txt":
            "Requisitos para participar en el programa Erasmus+\n\n"
            "1. Tener aprobados al menos 60 ECTS\n"
            "2. Nivel B1 del idioma del país de destino\n"
            "3. Estar matriculado en un grado o máster oficial"
    }
    
    for ruta, contenido in archivos.items():
        with open(ruta, "w", encoding="utf-8") as f:
            f.write(contenido)
    
    return list(archivos.keys())


# # Ejemplo de uso
# def caso_uso_dummy():
#     """
#     Demuestra un caso de uso completo con datos de ejemplo.
#     """
#     # Crear estructura de directorios y archivos dummy
#     print("Creando estructura de directorios y archivos de ejemplo...")
#     archivos_creados = crear_estructura_dummy()
#     print(f"Se han creado {len(archivos_creados)} archivos de ejemplo.\n")
    
#     # Inicializar el gestor de documentos
#     gestor = DocumentManager(db_path="normativa_upv_test.db")
#     print("Base de datos inicializada: normativa_upv_test.db\n")
    
#     # Indexar documentos iniciales
#     print("Indexando documentos iniciales...")
#     stats = gestor.procesar_directorio("documentos")
#     print(f"Resultado de la indexación inicial:")
#     print(f"- Documentos nuevos: {stats['nuevos']}")
#     print(f"- Documentos actualizados: {stats['actualizados']}")
#     print(f"- Documentos sin cambios: {stats['sin_cambios']}")
#     print(f"- Errores: {stats['errores']}\n")
    
#     # Listar documentos por categoría
#     categorias = gestor.obtener_categorias()
#     print(f"Categorías disponibles: {', '.join(categorias)}\n")
    
#     for categoria in categorias:
#         print(f"Documentos en la categoría '{categoria}':")
#         docs = gestor.buscar_documentos(categoria=categoria)
#         for doc in docs:
#             print(f"- {doc['nombre']} (última modificación: {doc['fecha_modificacion']})")
#         print()
    
#     # Modificar un documento existente
#     print("Modificando un documento existente...")
#     archivo_modificar = archivos_creados[0]
#     with open(archivo_modificar, "a", encoding="utf-8") as f:
#         f.write("\n\n3. Trabajo Fin de Máster\nEl TFM debe tener una extensión mínima de 50 páginas.")
    
#     print(f"Documento modificado: {archivo_modificar}")
    
#     # Actualizar solo la carpeta de normativa de másters
#     directorio_actualizar = "documentos/normativa_masters"
#     print(f"Actualizando documentos en: {directorio_actualizar}")
    
#     stats = gestor.procesar_directorio(directorio_actualizar, actualizar=True)
#     print(f"Resultado de la actualización:")
#     print(f"- Documentos nuevos: {stats['nuevos']}")
#     print(f"- Documentos actualizados: {stats['actualizados']}")
#     print(f"- Documentos sin cambios: {stats['sin_cambios']}")
#     print(f"- Errores: {stats['errores']}\n")
    
#     # Crear un nuevo documento
#     nuevo_documento = "documentos/normativa_masters/master_industrial.txt"
#     with open(nuevo_documento, "w", encoding="utf-8") as f:
#         f.write("Normativa del Máster en Ingeniería Industrial\n\n"
#                "1. Requisitos\nSe requiere grado en ingeniería industrial.\n\n"
#                "2. Especialidades\nAutomatización, Estructuras, Energía.")
    
#     print(f"Nuevo documento creado: {nuevo_documento}")
    
#     # Actualizar de nuevo
#     stats = gestor.procesar_directorio(directorio_actualizar, actualizar=True)
#     print(f"Resultado de la segunda actualización:")
#     print(f"- Documentos nuevos: {stats['nuevos']}")
#     print(f"- Documentos actualizados: {stats['actualizados']}")
#     print(f"- Documentos sin cambios: {stats['sin_cambios']}")
#     print(f"- Errores: {stats['errores']}\n")
    
#     # Generar informe
#     print("Generando informe de documentos:")
#     informe = gestor.generar_informe()
#     print(informe)
    
#     # Cerrar conexión a la BD
#     gestor.cerrar()
#     print("\nProceso completado. La base de datos está lista para ser utilizada en un sistema RAG.")


# if __name__ == "__main__":
#     caso_uso_dummy()


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: C:\Users\victo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: C:\Users\victo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Creando estructura de directorios y archivos de ejemplo...
Se han creado 4 archivos de ejemplo.

Base de datos inicializada: normativa_upv_test.db

Indexando documentos iniciales...
Resultado de la indexación inicial:
- Documentos nuevos: 1
- Documentos actualizados: 0
- Documentos sin cambios: 5
- Errores: 0

Categorías disponibles: erasmus_normativa, grados_normativa, normativa_masters

Documentos en la categoría 'erasmus_normativa':
- requisitos_erasmus.txt (última modificación: 2025-02-28T11:18:45.515350)

Documentos en la categoría 'grados_normativa':
- grado_ade.txt (última modificación: 2025-02-28T11:18:45.515350)
- grado_cienciaDatos.txt (última modificación: 2025-02-28T11:21:50.208417)

Documentos en la categoría 'normativa_masters':
- master_informatica.txt (última modificación: 2025-02-28T11:18:45.589585)
- master_teleco.txt (última modificación: 2025-02-28T11:18:45.515350)
- master_industrial.txt (última modificación: 2025-02-28T11:18:45.605084)

Modificando un documento ex


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: C:\Users\victo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import os

def list_directories():
    base_dir = "normativa_upv_test"
    
    print(f"Main directory: {base_dir}")
    if os.path.exists(base_dir):
        print("Subdirectories:")
        for subdir in os.listdir(base_dir):
            subdir_path = os.path.join(base_dir, subdir)
            if os.path.isdir(subdir_path):
                print(f"  - {subdir}")
                # List files in each subdirectory
                files = os.listdir(subdir_path)
                if files:
                    print("    Files:")
                    for file in files:
                        print(f"      - {file}")
                else:
                    print("    No files yet")
    else:
        print(f"Directory {base_dir} does not exist yet. Run the data collection functions first.")

# Call this function after your data collection
list_directories()

Main directory: normativa_upv_test
Directory normativa_upv_test does not exist yet. Run the data collection functions first.
